In [272]:
import json
import numpy as np
from pprint import pprint

fileName = '/Users/erichsu/LING406/yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json'

data = []
count = 0
with open(fileName) as f:
    for line in f:
        if count<100000:
            count += 1
            data.append(json.loads(line))

In [273]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [274]:
import nltk.data
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [275]:
sentences = []
print "Parsing sentences from training set"

for entry in data:
    sentences += review_to_sentences(entry['text'], tokenizer)


Parsing sentences from training set


In [276]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)



Training model...


In [277]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [278]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in data:
    clean_train_reviews.append( review_to_wordlist( review['text'], \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )


Review 0 of 100000
Review 1000 of 100000
Review 2000 of 100000
Review 3000 of 100000
Review 4000 of 100000
Review 5000 of 100000
Review 6000 of 100000
Review 7000 of 100000
Review 8000 of 100000
Review 9000 of 100000
Review 10000 of 100000
Review 11000 of 100000
Review 12000 of 100000
Review 13000 of 100000
Review 14000 of 100000
Review 15000 of 100000
Review 16000 of 100000
Review 17000 of 100000
Review 18000 of 100000
Review 19000 of 100000
Review 20000 of 100000
Review 21000 of 100000
Review 22000 of 100000
Review 23000 of 100000
Review 24000 of 100000
Review 25000 of 100000
Review 26000 of 100000
Review 27000 of 100000
Review 28000 of 100000
Review 29000 of 100000
Review 30000 of 100000
Review 31000 of 100000
Review 32000 of 100000
Review 33000 of 100000
Review 34000 of 100000
Review 35000 of 100000
Review 36000 of 100000
Review 37000 of 100000
Review 38000 of 100000
Review 39000 of 100000
Review 40000 of 100000
Review 41000 of 100000
Review 42000 of 100000
Review 43000 of 100000
R

In [279]:
stars = []

for entry in data:
    stars.append(entry['stars'])

label = np.array(stars, dtype='i')

In [280]:
labels = pd.DataFrame(label)

In [281]:
labels['rate'] =  pd.DataFrame(label)
for i in range(len(data)):
    if (labels[0][i]>3):
        labels['rate'][i] = 1
    else:
        labels['rate'][i] = 0

In [282]:
bad = []

for i in range(len(data)):
    if np.isnan(trainDataVecs[i]).any():
        bad.append(i)
        print "true"

true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true


In [283]:
trainSet = pd.DataFrame(trainDataVecs)

In [284]:
trainSet['rate'] = labels['rate']

In [285]:
trainSet = trainSet.dropna()

In [289]:
trainSet = trainSet.reset_index()

In [296]:
train = trainSet.iloc[0:80000]
test = trainSet.iloc[80000:100000]

In [297]:
# Fit a random forest to the training data, using 100 trees
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print "Fitting a random forest to labeled training data..."
forest = forest.fit(train[train.columns[0:299]], train['rate'] )




Fitting a random forest to labeled training data...


In [298]:
result = forest.predict( test[test.columns[0:299]] )

In [300]:
ok = pd.DataFrame()
ok['target'] = test['rate']
ok = ok.reset_index()
no = pd.DataFrame()
no = pd.DataFrame(result)
ok['res'] = no[0]
ok['t'] = ok['target'] - ok['res']

count = 0
for i in range(len(result)):
    if(ok['t'][i]!=0):
        count += 1

In [301]:
count

3524

In [265]:
no = pd.DataFrame()
no = pd.DataFrame(result)

In [266]:
len(no)

9993

In [267]:
ok['res'] = no[0]

In [268]:
ok['t'] = ok['target'] - ok['res']

In [271]:
for i in range(len(result)):
    if(ok['t'][i]!=0):
        print "aww"

In [234]:
result

array([1, 0, 1, ..., 1, 1, 1], dtype=int32)

In [235]:
ok['target']

0     1
1     0
2     1
3     1
4     1
5     0
6     1
7     1
8     0
9     1
10    1
11    0
12    0
13    1
14    0
...
9985    1
9986    1
9987    1
9988    1
9989    1
9990    1
9991    0
9992    1
9993    0
9994    1
9995    0
9996    1
9997    1
9998    1
9999    1
Name: target, Length: 9993, dtype: int32

In [239]:
len(trainSet['rate'] )

9993